In [71]:
from sklearn.model_selection import train_test_split,cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor,HistGradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb

In [72]:
match_data = pd.read_csv('..\\Data\\matchData2.csv')
wins = pd.read_csv('..\\Data\\matchDataWins.csv')

In [73]:
# Realizo el split de los datos con un random state de 42 para que siempre se divida igual
predictors_train, predictors_test, wins_team1_train, wins_team1_test, wins_team2_train, wins_team2_test = train_test_split(predictors, win_team1, win_team2, test_size=0.2, random_state=42)

# Transformo los datos a formato csv
predictors_train.to_csv('..\\Data\\predictors_train.csv', index=False)
predictors_test.to_csv('..\\Data\\predictors_test.csv', index=False)
wins_team1_train.to_csv('..\\Data\\wins_team1_train.csv', index=False)
wins_team1_test.to_csv('..\\Data\\wins_team1_test.csv', index=False)
wins_team2_train.to_csv('..\\Data\\wins_team2_train.csv', index=False)
wins_team2_test.to_csv('..\\Data\\wins_team2_test.csv', index=False)

In [65]:
match_data = match_data.drop(['firstBaronTeam1', 'firstBaronTeam2', 'win_team1', 'win_team2'], axis=1)
predictors = match_data
win_team1 = pd.DataFrame(wins['win_team1'])
win_team2 = pd.DataFrame(wins['win_team2'])

In [66]:
# Modelos utilizados
RF_classifier = RandomForestClassifier()
RF_regressor = RandomForestRegressor()
B_classifier = xgb.XGBClassifier()
B_regressor = HistGradientBoostingRegressor()

In [57]:
# Aplicamos Cross Validation a los modelos
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Realizar la validación cruzada y obtener los puntajes de rendimiento para la predicción de partidas
RF_scores_win = cross_val_score(RF_classifier, predictors, wins, cv=kfold, n_jobs=-1)
B_scores_win1 = cross_val_score(B_classifier, predictors, win_team1, cv=kfold, n_jobs=-1)
B_scores_win2 = cross_val_score(B_classifier, predictors, win_team2, cv=kfold, n_jobs=-1)


# Imprimir los resultados
print("Puntajes de Validación Cruzada en RF:", RF_scores_win)
print("Precisión Promedio en RF: {:.2f}%".format(100 * RF_scores_win.mean()))
print("Puntajes de Validación Cruzada en Boosting con victorias del equipo 1:", B_scores_win1)
print("Precisión Promedio en Boosting con victorias del equipo 1: {:.2f}%".format(100 * B_scores_win1.mean()))
print("Puntajes de Validación Cruzada en Boosting con victorias del equipo 2:", B_scores_win2)
print("Precisión Promedio en Boosting con victorias del equipo 2: {:.2f}%".format(100 * B_scores_win2.mean()))

Puntajes de Validación Cruzada en RF: [0.74700808 0.74660163 0.75053064 0.74371133 0.74342878]
Precisión Promedio en RF: 74.63%
Puntajes de Validación Cruzada en Boosting con victorias del equipo 1: [0.75351127 0.75608544 0.75734995 0.75184031 0.75467437]
Precisión Promedio en Boosting con victorias del equipo 1: 75.47%
Puntajes de Validación Cruzada en Boosting con victorias del equipo 2: [0.75351127 0.75608544 0.75734995 0.75184031 0.75467437]
Precisión Promedio en Boosting con victorias del equipo 2: 75.47%


In [58]:
game_duration = match_data['gameDuration']
match_data = match_data.drop(['gameDuration'], axis=1)
predictors = match_data

In [59]:
# Aplicamos Cross Validation a los modelos para la predicción de la duración de las partidas
RF_scores_time = cross_val_score(RF_regressor, predictors, game_duration, cv=kfold, n_jobs=-1, scoring='neg_mean_squared_error')
B_scores_time = cross_val_score(B_regressor, predictors, game_duration, cv=kfold, n_jobs=-1, scoring='neg_mean_squared_error')

# Imprimir los resultados
print("Puntajes de Validación Cruzada en RF:", -1 * RF_scores_time)
print("Raíz cuadrada media Promedio en RF: {:.2f}".format(-1 * RF_scores_time.mean()))
print("Puntajes de Validación Cruzada en Boosting:", -1 * B_scores_time)
print("Raíz cuadrada media Promedio en Boosting: {:.2f}".format(-1 * B_scores_time.mean()))

Puntajes de Validación Cruzada en RF: [114321.00357182 115701.16829645 115418.82430958 114150.6218106
 113575.82747411]
Raíz cuadrada media Promedio en RF: 114633.49
Puntajes de Validación Cruzada en Boosting: [111854.4188323  113049.58511378 112576.81527083 111944.69104756
 110922.52557155]
Raíz cuadrada media Promedio en Boosting: 112069.61


In [70]:
# Buscamos los mejores parametros para el modelo de Random Forest para clasificacion ()
param_distributions = { 
    'n_estimators': randint(50, 500),
    'max_depth': randint(1, 20),
    'max_features': randint(1, 20),
    'min_samples_leaf': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(RF_classifier, param_distributions=param_distributions, n_iter=20, cv=2, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(predictors, wins)
random_search.best_params_

Fitting 2 folds for each of 20 candidates, totalling 40 fits


{'bootstrap': True,
 'max_depth': 15,
 'max_features': 11,
 'min_samples_leaf': 8,
 'min_samples_split': 8,
 'n_estimators': 171}